In [3]:
# General imports 
import os
import cv2
import json
import numpy as np

In [9]:
# Define directories 
home_dir = '/Users/sinakling/disks/meso_shared'
main_dir = '{}/centbids'.format(home_dir)
project_dir = 'centbids'

tasks = ['pRF']
task = tasks[0]
stim_video_fn = '{}/derivatives/vdm/{}_vid(4).mp4'.format(main_dir, task)

# Analysis parameters
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../../../../"))
settings_path = os.path.join(base_dir, project_dir, "settings.json")

with open(settings_path) as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

In [ ]:
# open video
cap = cv2.VideoCapture(stim_video_fn)

# get video settings
vid_width, vid_height = cap.get(3), cap.get(4)
vid_fps = cap.get(cv2.CAP_PROP_FPS)
vid_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# set VDM settings
TR = analysis_info['TR']

vdm_width = analysis_info['vdm_size_pix'][0] 
vdm_height = analysis_info['vdm_size_pix'][1]

vdm_frames_sampled = vid_fps * TR
vdm_frames = vid_frames / vdm_frames_sampled
vdm_mat = np.zeros((vdm_width, vdm_height, int(vdm_frames)))


In [ ]:
# define top and bottom part as 16/9 screen experiment and VDM should be a square: 

#=============== Screen ===============================
#               1920 px  (~73.3 cm)                                                      ============ Converting rectangular screen to square ============
# +--------------------------------------------------+												
# |                                                  |
# |                                                  |													1920 px  (~73.3 cm)
# |                                                  |									  +--------------------------------------------------+
# |                                                  |	1080 px  (~43.5 cm)  ->			  |                  MARGIN                          |
# +--------------------------------------------------+					        		  |              (420 px each)                       |
#																						  +--------------------------------------------------+
#																						  |                                                  |
#																						  |                SQUARE AREA                       |
#																						  |               1920 x 1920 px                     |
#																						  |                                                  |
#																						  +--------------------------------------------------+
#																						  |                  MARGIN                          |
#																						  |              (420 px each)                       |
#																						  +--------------------------------------------------+
#																						
#																						  Screen height = 1080 px
#																						  Square size   = 1920 px
#																						  Added margins = (1920 - 1080) / 2 = 420 px (top & bottom). Final dimensions, to be taken into account when giving dimensions to prfpy!



height_to_add = (vid_width - vid_height) / 2 
add_mat =  np.zeros((int(height_to_add),int(vid_width)))

In [14]:
vdm_video_fn = '{}/derivatives/vdm/vdm_{}_{}_{}.mp4'.format(main_dir, task, vdm_width, vdm_height)
vdm_numpy_fn = '{}/derivatives/vdm/vdm_{}_{}_{}.npy'.format(main_dir, task, vdm_width, vdm_height)

In [15]:
# create VDM matrix
vid_idx, vdm_idx = 0, 0
while(cap.isOpened()):
    
    # read video frames
    ret, vid_frame = cap.read()
    if not ret: break

    # process sampled frames
    if np.mod(vid_idx, int(vdm_frames_sampled)) == 0:

        # convert frame to grayscale
        gray_mat = cv2.cvtColor(vid_frame, cv2.COLOR_BGR2GRAY)

        # convert to binary
        binary_mat = (gray_mat > 5).astype(np.uint8)
    
        # add top and bottom blank    
        binary_reshape_mat = np.concatenate((add_mat,binary_mat,add_mat), axis=0)
        
        # resize to create VDM
        binary_reshape_resize_mat = cv2.resize(binary_reshape_mat, dsize=(vdm_width, vdm_height),
                                               interpolation=cv2.INTER_NEAREST)
        
        # fill VDM matrix
        vdm_mat[...,vdm_idx] = binary_reshape_resize_mat
        vdm_idx += 1


    vid_idx += 1
    
cap.release()

In [ ]:
import numpy as np

# find coordinates of white pixels
white_coords = np.argwhere(vdm_mat == 1)

# get height (rows) and width (columns) of white pixels
white_heights = white_coords[:, 0]
white_widths = white_coords[:, 1]

print("White pixel heights (row indices):", white_heights)
print("White pixel widths (column indices):", white_widths)



# ===== Stimulus grid ==========
#
#   0        18       32       50   (px)
#   |--------|--------|--------|
#   |        |        |        |
#   |        +--------+        |
#   |        | 15x15  |        |
#   |        |  box   |        |
#   |        +--------+        |
#   |                          |
#   +--------------------------+
#  0                          50
#        (50 x 50 px matrix)



White pixel heights (row indices): [18 18 18 ... 32 32 32]
White pixel widths (column indices): [18 18 18 ... 32 32 32]


In [16]:
# save VDM as numpy matrix
np.save(vdm_numpy_fn, vdm_mat)

# save VDM as video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(vdm_video_fn, fourcc, 1/TR, (vdm_width, vdm_height), False)
[out.write(np.uint8(frame*255)) for frame in np.split(vdm_mat, vdm_frames, axis=2)]
out.release()